## Overview

This notebook:

- creates a MessageHub instance in Bluemix
- produces some messages using vanilla python
- consumes those messages using vanilla python
- produces some more messages using vanilla python
- consumes those messages using scala spark streaming (requires step 9)
- tears down the MessageHub instance

Enter your bluemix ID

In [1]:
from getpass import getpass
ibm_id = raw_input("IBM ID: ")

IBM ID: chris.snow@uk.ibm.com


Enter your bluemix ID password

In [2]:
from getpass import getpass
ibm_id_password = getpass("Password: ")

Password: ········


Change these variables for your environment

In [4]:
# change this to point to your bluemix organization name
bluemix_organization_name = 'chris.snow@uk.ibm.com'

# change this to point to your bluemix space name
bluemix_space_name = 'dev'

# You may need to change the target_endpoint to:
#
#   https://api.ng.bluemix.net     - for the US South Region
#   https://api.eu-gb.bluemix.net  - for the UK
#   https://api.au-syd.bluemix.net - for Australia

target_endpoint = 'https://api.ng.bluemix.net'

You may want to change these

In [5]:
# This is the name of the Message Hub service instance that will
# be created for you in Bluemix
messagehub_instance_name = 'my_messagehub'

# This is the name of a topic that will get created for you
messagehub_topic_name = 'my_topic'

# Do you want to delete any existing MessageHub instance with
# messagehub_instance_name for before creating a new one?
delete_messagehub_instance = True

Install a python utility script from github for interacting with cloud foundry via the API

In [6]:
!pip install --user --upgrade --quiet git+https://github.com/snowch/cf-python-client
!pip install --user --upgrade --quiet git+https://github.com/snowch/nb_utils

print version of installed nb_utils for traceability purposes

In [7]:
import requests, json
requests.get('https://api.github.com/repos/snowch/nb_utils/git/refs/heads/master').json()

{u'object': {u'sha': u'04d1ae56e6093a2ac52818661bd8679df2a36c9b',
  u'type': u'commit',
  u'url': u'https://api.github.com/repos/snowch/nb_utils/git/commits/04d1ae56e6093a2ac52818661bd8679df2a36c9b'},
 u'ref': u'refs/heads/master',
 u'url': u'https://api.github.com/repos/snowch/nb_utils/git/refs/heads/master'}

Create a python object for interacting with cloud foundry

In [8]:
from mh_utils import mh_utils
mh = mh_utils.MessageHubUtil(target_endpoint, ibm_id, ibm_id_password, bluemix_organization_name, bluemix_space_name)

Remove any MessageHub instances with the messagehub_instance_name

In [9]:
if delete_messagehub_instance:
    mh.delete_service_instance(messagehub_instance_name, True)

Create a MessageHub service instance
```
mh_service_plan_guid = 'fe959ac5-aa47-43a6-9c58-6fc265ee9b0e'

# This is the guid of the Message Hub service plan. I found this using:
#
#   cf = CloudFoundryUtil(target_endpoint, ibm_id, ibm_id_password, bluemix_org_name, bluemix_space_name)
#   print(cf.search_plans('message hub'))
```

In [10]:
mh_service_plan_guid = 'fe959ac5-aa47-43a6-9c58-6fc265ee9b0e'
mh.create_service_instance(mh_service_plan_guid, messagehub_instance_name)

Extract some variables that we will need to work with the MessageHub instance

In [15]:
with open('messagehub.properties', 'w') as w:
    w.write(mh.get_properties(messagehub_instance_name))
    w.write('messagehub_topic_name={0}'.format(messagehub_topic_name))

Create a topic in the MessageHub instance

In [12]:
mh.create_topic(messagehub_instance_name, messagehub_topic_name)
mh.create_topic(messagehub_instance_name, messagehub_topic_name + "_responses")

[{"name":"my_topic","partitions":1,"retentionMs":"86400000","markedForDeletion":false}]
[{"name":"my_topic","partitions":1,"retentionMs":"86400000","markedForDeletion":false},{"name":"my_topic_responses","partitions":1,"retentionMs":"86400000","markedForDeletion":false}]


Send some messages

In [13]:
mh.produce_message(messagehub_instance_name, messagehub_topic_name, b'some_raw_bytes_1')
mh.produce_message(messagehub_instance_name, messagehub_topic_name, b'some_raw_bytes_1')

In [14]:
mh.consume_messages(messagehub_instance_name, messagehub_topic_name)

my_topic:0:0: key=None value=some_raw_bytes_1
my_topic:0:1: key=None value=some_raw_bytes_1


## Send prediction requests to MessageHub for spark streaming

Now open the notebook **"Step 9 - Scala spark streaming on DSX"** in another browser window.<br>
Run the notebook and while it is running, put some more messages in kafka by running the cell below.

### STEP 08 (A) - Produce Prediction Requests

In [25]:
mh.produce_message(messagehub_instance_name, messagehub_topic_name, '1,500') # user_id | movie_id
mh.produce_message(messagehub_instance_name, messagehub_topic_name, '1,501') # user_id | movie_id
mh.produce_message(messagehub_instance_name, messagehub_topic_name, '1,502') # user_id | movie_id
mh.produce_message(messagehub_instance_name, messagehub_topic_name, '1,503') # user_id | movie_id
mh.produce_message(messagehub_instance_name, messagehub_topic_name, '1000003,503') # user_id | movie_id : invalid user

The step 09 or 10 notebook should have made some predictions and put them on the responses topic.<br>
Run the following cell to retrieve the responses from kafka.

### STEP 08 (B) - Consume Prediction Responses

In [26]:
mh.consume_messages(messagehub_instance_name, messagehub_topic_name + "_responses")

my_topic_responses:0:0: key=None value=1, 500, 3.715163746095887
my_topic_responses:0:1: key=None value=1, 501, 3.1018965193747663
my_topic_responses:0:2: key=None value=1, 502, 1.9472057011680621
my_topic_responses:0:3: key=None value=1, 503, 2.6929616601709423
my_topic_responses:0:4: key=None value=1000003, 503, -1
my_topic_responses:0:5: key=None value=1, 500, 3.715163746095887
my_topic_responses:0:6: key=None value=1, 501, 3.1018965193747663
my_topic_responses:0:7: key=None value=1, 502, 1.9472057011680621
my_topic_responses:0:8: key=None value=1, 503, 2.6929616601709423
my_topic_responses:0:9: key=None value=1000003, 503, -1
my_topic_responses:0:10: key=None value=1, 500, 3.715163746095887
my_topic_responses:0:11: key=None value=1, 501, 3.1018965193747663
my_topic_responses:0:12: key=None value=1, 502, 1.9472057011680621
my_topic_responses:0:13: key=None value=1, 503, 2.6929616601709423
my_topic_responses:0:14: key=None value=1000003, 503, -1
my_topic_responses:0:15: key=None valu